In [1]:
# quick_verify_form_fixed_v2.py — 20%/50% 밴드 라벨 + 지도 저장 없이 즉시 시각화
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point

# --- 상수 ---
LOW20 = "low20_grids.geojson"   # 하위 20% 격자 (low20 ⊂ low50 가정)
LOW50 = "low50_grids.geojson"   # 하위 50% 격자 (20% 포함)
OUTDIR = "verify_maps"          # (사용하지 않지만 남겨둠)

# 전역 (로드 후 채움)
L20 = None   # GeoDataFrame: 하위 20%
L50 = None   # GeoDataFrame: 하위 50%
BANDS = None # L20 ∪ L50 (중복 제거)

# --- 데이터 로드 ---
def load_layers():
    global L20, L50, BANDS

    if not (os.path.exists(LOW20) and os.path.exists(LOW50)):
        print("필요 파일이 없습니다: low20_grids.geojson, low50_grids.geojson")
        return False

    l20 = gpd.read_file(LOW20).to_crs(4326).copy()
    l50 = gpd.read_file(LOW50).to_crs(4326).copy()

    # 필수 컬럼 확인
    need = {"grid_id", "final_score", "geometry"}
    if not need.issubset(l20.columns) or not need.issubset(l50.columns):
        print("GeoJSON에 grid_id, final_score, geometry 컬럼이 필요합니다.")
        return False

    # percentile 보강(없으면 rank_pct로부터)
    if "percentile" not in l20.columns:
        l20["percentile"] = (l20.get("rank_pct", np.nan) * 100).round(1)
    if "percentile" not in l50.columns:
        l50["percentile"] = (l50.get("rank_pct", np.nan) * 100).round(1)

    # 레이어 라벨
    l20 = l20.assign(band_label="20% 이내")
    l50 = l50.assign(band_label="50% 이내")

    # 전역 저장
    L20 = l20[["grid_id","final_score","percentile","band_label","geometry"]].copy()
    L50 = l50[["grid_id","final_score","percentile","band_label","geometry"]].copy()

    # 유니온 (중복 grid_id 제거)
    BANDS = pd.concat([L50, L20], ignore_index=True).drop_duplicates(subset=["grid_id"]).reset_index(drop=True)
    return True

# --- 판정 + 출력 ---
def verify_once(lat: float, lon: float, name: str = ""):
    """
    입력 좌표가:
      - 20% 이내(L20 교차)면 → '20% 이내'
      - 아니고 50% 이내(L50 교차)면 → '50% 이내'
      - 둘 다 아니면 → '안된다'
    를 출력하고, 도봉구 전체(=저득점 격자 전체 영역) 위에서 위치/격자를 즉시 시각화.
    """
    pt = gpd.GeoDataFrame({"name":[name]}, geometry=[Point(lon, lat)], crs=4326)

    # 1) 20% 이내 여부 먼저 (low50이 low20을 포함하므로 우선 판정)
    hit20 = gpd.sjoin(pt, L20, how="left", predicate="intersects").drop(columns="index_right", errors="ignore")
    in20 = pd.notna(hit20.loc[0, "grid_id"])

    # 2) 50% 이내 여부
    hit50 = gpd.sjoin(pt, L50, how="left", predicate="intersects").drop(columns="index_right", errors="ignore")
    in50 = pd.notna(hit50.loc[0, "grid_id"])

    # 3) 결과 텍스트
    print(f"\n[{name or '-'}] ({lat:.6f}, {lon:.6f})")
    if in20:
        row = hit20.iloc[0]
        band_str = "20% 이내"
    elif in50:
        row = hit50.iloc[0]
        band_str = "50% 이내"
    else:
        print("50% 이내 해당 없음")  # 50% 이내 해당 없음
        # 그래도 위치가 어디쯤인지 한눈에 보이도록 전체 레이어와 함께 시각화
        m = build_overview_map(lat, lon, highlight_gid=None, band_str=None)
        _display_map(m)
        return

    gid = int(row["grid_id"])
    pct = float(row["percentile"]) if pd.notna(row["percentile"]) else None
    score = float(row["final_score"]) if pd.notna(row["final_score"]) else None

    print(f"- Band       : {band_str}")
    print(f"- Grid ID    : {gid}")
    print(f"- FinalScore : {score:.3f}" if score is not None else "- FinalScore : NA")
    print(f"- Percentile : {pct:.1f}%" if pct is not None else "- Percentile : NA")

    # 4) 시각화 (전체 도봉구 저득점 격자 위에, 해당 격자만 굵게 하이라이트)
    m = build_overview_map(lat, lon, highlight_gid=gid, band_str=band_str)
    _display_map(m)

# --- 지도 빌드 (저장 없이 화면에 바로 표시) ---
def build_overview_map(lat, lon, highlight_gid=None, band_str=None) -> folium.Map:
    # 중심과 초기 줌
    if len(L50) > 0:
        minx, miny, maxx, maxy = L50.total_bounds
        center = [(miny + maxy) / 2, (minx + maxx) / 2]
    else:
        center = [lat, lon]

    m = folium.Map(location=center, zoom_start=13, tiles="cartodbpositron")

    # 하위 50% 전체(연한색)
    folium.GeoJson(
        L50.__geo_interface__,
        name="하위 50% (연한 주황)",
        style_function=lambda f: {"fillColor":"#ffedcc","color":"#b3b3b3","weight":0.7,"fillOpacity":0.35},
        tooltip=folium.GeoJsonTooltip(fields=["grid_id","final_score","percentile"],
                                      aliases=["Grid ID","Final","Percentile(%)"], localize=True)
    ).add_to(m)

    # 하위 20% (진한 빨강)
    folium.GeoJson(
        L20.__geo_interface__,
        name="하위 20% (붉은색)",
        style_function=lambda f: {"fillColor":"#ff0000","color":"#6b0000","weight":0.8,"fillOpacity":0.45},
        tooltip=folium.GeoJsonTooltip(fields=["grid_id","final_score","percentile"],
                                      aliases=["Grid ID","Final","Percentile(%)"], localize=True)
    ).add_to(m)

    # 입력 포인트
    folium.CircleMarker([lat, lon], radius=6, color="#000", fill=True, fill_opacity=1,
                        popup=f"입력좌표 ({lat:.6f}, {lon:.6f})").add_to(m)

    # 해당 격자만 굵게 하이라이트(있다면)
    if highlight_gid is not None:
        poly = BANDS.loc[BANDS["grid_id"] == highlight_gid]
        if len(poly) > 0:
            hl_color = "#d10000" if band_str == "20% 이내" else "#ff9900"
            folium.GeoJson(
                poly.__geo_interface__,
                name=f"선택 격자 (Grid {highlight_gid})",
                style_function=lambda f, c=hl_color: {"fillColor":c,"color":"#000","weight":2.2,"fillOpacity":0.15}
            ).add_to(m)
            # 해당 격자 bbox에 맞춰 보기
            minx, miny, maxx, maxy = poly.total_bounds
            m.fit_bounds([[miny, minx], [maxy, maxx]])
        else:
            # 격자 못 찾으면 L50 전체에 맞춤
            minx, miny, maxx, maxy = L50.total_bounds
            m.fit_bounds([[miny, minx], [maxy, maxx]])
    else:
        # outside인 경우: 전체 저득점 영역 기준으로 맞춤
        minx, miny, maxx, maxy = L50.total_bounds
        m.fit_bounds([[miny, minx], [maxy, maxx]])

    folium.LayerControl(collapsed=False).add_to(m)
    return m

def _display_map(m: folium.Map):
    """주피터/Lab이면 지도를 바로 띄움. (파일 저장/브라우저 오픈 없이)"""
    try:
        from IPython.display import display  # noqa
        display(m)
    except Exception:
        print("(참고) 현재 환경에서는 지도 미리보기를 바로 띄울 수 없습니다. Jupyter/Lab 권장.")

# --- 메인 ---
def run_verification():
    print("=== 좌표 한 번 입력해서 검증 (20%/50% 이내 구분 + 즉시 시각화) ===")
    lat_str = input("위도(lat): ").strip()
    lon_str = input("경도(lon): ").strip()

    if not lat_str or not lon_str:
        print("\n[오류] 위도와 경도를 모두 입력해야 합니다.")
        return

    try:
        lat = float(lat_str); lon = float(lon_str)
    except ValueError:
        print("\n[오류] 위도와 경도는 숫자여야 합니다.")
        return

    name = input("이름(선택): ").strip()
    verify_once(lat, lon, name)

if __name__ == "__main__":
    if load_layers():
        run_verification()


=== 좌표 한 번 입력해서 검증 (20%/50% 이내 구분 + 즉시 시각화) ===

[5] (22.222000, 2.555000)
50% 이내 해당 없음


In [2]:
print('a')

a
